In [1]:
def mountFileSystem(containerName, storageAccountName):
  configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": dbutils.secrets.get(scope = "kvbenchmark", key = "sp-adls-appid"),
       "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope = "kvbenchmark", key = "sp-adls-secret"),
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/microsoft.onmicrosoft.com/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}
  
  mountPoint = "/mnt/adls/" + storageAccountName + "/" + containerName
  try:
    dbutils.fs.mount(
      source = "abfss://" + containerName + "@" + storageAccountName + ".dfs.core.windows.net",
      mount_point = mountPoint,
      extra_configs = configs
    )
    print(mountPoint + " mounted successfully")
  except:
    print("The mount " + mountPoint + " already exists.")

  return mountPoint


In [2]:
mountFileSystem("tpcds1tb", "adls7dataset7benchmark")

In [3]:
import os
parquet_path = "/mnt/adls/adls7dataset7benchmark/tpcds1tb/parquet/"

In [4]:
server_name = dbutils.secrets.get(scope = "kvbenchmark", key = "db-server-url")
database_name = dbutils.secrets.get(scope = "kvbenchmark", key = "db-name-noidx")
url = server_name + ";" + "databaseName=" + database_name + ";"

username = dbutils.secrets.get(scope = "kvbenchmark", key = "db-username")
password = dbutils.secrets.get(scope = "kvbenchmark", key = "db-password")

In [5]:
table_name = "dbo.store"
df = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password) \
        .load()

df.write.parquet(os.path.join(parquet_path, "store"), mode="overwrite")

In [6]:
table_name = "dbo.store_sales"
df = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()

df.write.parquet(os.path.join(parquet_path, "store_sales"), mode="overwrite", partitionBy="ss_store_sk")

In [7]:
table_name = "dbo.store_returns"
df = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()

df.write.parquet(os.path.join(parquet_path, "store_returns"), mode="overwrite", partitionBy="sr_store_sk")

In [8]:
table_name = "dbo.customer"
df = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()

df.write.parquet(os.path.join(parquet_path, "customer"), mode="overwrite")